In [33]:
import pandas as pd
import numpy as np
import glob
import os
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.utils import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
import scipy

import tensorflow as tf

from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
from tensorflow.keras.preprocessing.image import img_to_array, load_img, ImageDataGenerator
from tensorflow.keras.layers import Activation, Dense, GlobalAveragePooling2D, GlobalMaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
import tensorflow.keras.backend as K

#from nfnets_keras import NFNetF1

#salmon-scales
from train_util import read_images, load_xy, get_checkpoint_tensorboard, create_model_grayscale, get_fresh_weights, base_output, dense1_linear_output, train_validate_test_split


In [34]:
!nvidia-smi

Wed Oct  6 07:50:10 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.73.01    Driver Version: 460.73.01    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:1A:00.0 Off |                  N/A |
| 40%   63C    P0    78W / 250W |      0MiB / 11019MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  RTX A6000           Off  | 00000000:67:00.0 Off |                  Off |
| 40%   

In [35]:
def base_output(model):
    z = model.output
    z = GlobalMaxPooling2D()(z)
    return z

def dense1_linear_output(gray_model):
    z = base_output(gray_model)
    z = Dense(1, activation='linear')(z)
    return z

def get_checkpoint_tensorboard(tensorboard_path, checkpoint_path):
    
    tensorboard = TensorBoard(log_dir=tensorboard_path)
    checkpointer = ModelCheckpoint(
        filepath = checkpoint_path,
        verbose = 1,
        save_best_only = True,
        save_weights_only = False)
    return tensorboard, checkpointer

def train_validate_test_split(pairs, validation_set_size = 0.15, test_set_size = 0.15, a_seed = 8):
    """ split pairs into 3 set, train-, validation-, and test-set
        1 - (validation_set_size + test_set_size) = % training set size
    >>> import pandas as pd
    >>> import numpy as np
    >>> data = np.array([np.arange(10)]*2).T  # 2 columns for x, y, and one for index
    >>> df_ = pd.DataFrame(data, columns=['x', 'y'])
    >>> train_x, val_x, test_x = \
             train_validate_test_split( df_, validation_set_size = 0.2, test_set_size = 0.2, a_seed = 1 )
    >>> train_x['x'].values
    array([0, 3, 1, 7, 8, 5])
    >>> val_x['x'].values
    array([4, 6])
    >>> test_x['x'].values
    array([2, 9])
    """
    validation_and_test_set_size = validation_set_size + test_set_size
    validation_and_test_split = validation_set_size / (test_set_size+validation_set_size)
    df_train_x, df_notTrain_x = train_test_split(pairs, test_size = validation_and_test_set_size, random_state = a_seed)
    df_test_x, df_val_x = train_test_split(df_notTrain_x, test_size = validation_and_test_split, random_state = a_seed)
    return df_train_x, df_val_x, df_test_x

In [126]:
os.listdir( '/gpfs/gpfs0/deep/data/Savannah_Professional_Practice2021_06_10_21/CodOtholiths-MachineLearning/Savannah_Professional_Practice/2013/70021/Nr09_age02' )

['IMG_0160.JPG',
 'IMG_0161.JPG',
 'IMG_0159.JPG',
 'IMG_0157.JPG',
 'IMG_0162.JPG',
 'IMG_0158.JPG']

In [77]:
os.listdir('/gpfs/gpfs0/deep/data/Savannah_Professional_Practice2021_06_10_21/CodOtholiths-MachineLearning/Savannah_Professional_Practice')

['Extra']

In [119]:
tmp = 'Nr 02 age_09'
a = tmp.find('age')
tmp[0:6]
tmp2 = 'Nr01_age07'

'Nr 02 '

In [ ]:
from PIL import Image, ExifTags

#def read_jpg_cods(B4_input_shape = (380, 380, 3), max_dataset_size = 5180, loadImgToMem=False):
#    '''
#    reads one .jpg file in each folder in structure of folders
#    returns tensor with images, and 1-1 correspondence with age
#    '''

max_dataset_size = 5180
B4_input_shape = (380, 380, 3)
df_cod = pd.DataFrame(columns=['age', 'path', 'ExposureTime'])
image_tensor1 = np.empty(shape=(max_dataset_size,)+B4_input_shape)
image_tensor2 = np.empty(shape=(max_dataset_size,)+B4_input_shape)
image_tensor3 = np.empty(shape=(max_dataset_size,)+B4_input_shape)

base_dir = '/gpfs/gpfs0/deep/data/Savannah_Professional_Practice2021_06_10_21/CodOtholiths-MachineLearning/Savannah_Professional_Practice'
df_cod = pd.DataFrame(columns=['age', 'path', 'ExposureTime'])
base_dirs_posix = Path(base_dir)

error_count=0
add_count = 0
for some_year_dir in base_dirs_posix.iterdir():
    count = 0
    if not os.path.isdir( some_year_dir ):
        continue
    
    #dir structure: /year/station_number/cod_img_by_age/6 jpeg images of one fish
    stat_nos = [name for name in os.listdir( some_year_dir ) if os.path.isdir(os.path.join(some_year_dir , name))]
    for i in range(0, len(stat_nos)):
        cod_path = os.path.join( some_year_dir, stat_nos[i] )
        yr_station_codage_path = [os.path.join(cod_path , n) for n in os.listdir( cod_path ) 
                        if os.path.isdir(os.path.join(cod_path , n))]
        cod_age = [n for n in os.listdir( cod_path ) 
                        if os.path.isdir(os.path.join(cod_path , n))]
        
        assert len(yr_station_codage_path) == len(cod_age)
        for j in range(0, len(yr_station_codage_path)):
            #print(onlyfiles)
            onlyfiles = [f for f in os.listdir( yr_station_codage_path[j] ) 
                         if os.path.isfile(os.path.join(yr_station_codage_path[j] , f))]
            if len(onlyfiles) != 6: 
                if 'Thumbs.db' in onlyfiles and len(onlyfiles)==7:
                   continue
                #print(str(len(onlyfiles)) + '\t' + str( yr_station_codage_path[j] ) + "\t" +'\t'.join(map(str,onlyfiles)))
                error_count +=1
            else: 
                
                full_path = [os.path.join(yr_station_codage_path[j] , f) 
                         for f in os.listdir( yr_station_codage_path[j] ) 
                     if os.path.isfile(os.path.join(yr_station_codage_path[j] , f))]

                begin_age = cod_age[j].lower().find('age')
                #print(cod_age[j])
                age = cod_age[j][begin_age+3:begin_age+5]
                try:
                    age = int(age)
                except ValueError:
                    #print(yr_station_codage_path[j])
                    #print(cod_age[j])
                    #print(age)
                    #print(begin_age)
                    age = 0
                
                #print(age)
                
                full_path.sort()
                exposures = set()
                for k in range(0, len(full_path)): #len(full_path) == 6
                    img = Image.open(full_path[k])
                    exif = {ExifTags.TAGS[k]: v for k, v in img._getexif().items() if k in ExifTags.TAGS}
                    #print(exif['ExposureTime'])
                    exposures.add(exif['ExposureTime'])
                
                
                if len(exposures) != 3:
                    #print("***")
                    exposures = []
                    for k in range(0, len(full_path)):
                        img = Image.open(full_path[k])
                        exif = {ExifTags.TAGS[k]: v for k, v in img._getexif().items() if k in ExifTags.TAGS}
                        exposures.append( exif['ExposureTime'] )
                        
                    print(str (yr_station_codage_path[j] ) + '\t' + str(exposures) ) 
                else:
                    exposures = []
                    numpy_images = [0,0,0]
                    file_paths = [0,0,0]
                    imgs_added = 0
                    for k in range(0, len(full_path)):
                        img = Image.open(full_path[k])
                        exif = {ExifTags.TAGS[k]: v for k, v in img._getexif().items() if k in ExifTags.TAGS}
                        if not exif['ExposureTime'] in exposures:
                            exposures.append( exif['ExposureTime'] )
                            
                            pil_img = load_img(full_path[k], target_size=B4_input_shape, grayscale=False)
                            array_img = img_to_array(pil_img, data_format='channels_last')
                            
                            numpy_images[imgs_added] = array_img
                            file_paths[imgs_added] = full_path[k]
                            imgs_added += 1
                    
                    expo_args = np.argsort(exposures).tolist()
                    print(str(exposures) + " :" + str(file_paths))
                    print(expo_args)
                    
                    image_tensor1[add_count] = numpy_images[expo_args[0]]
                    image_tensor2[add_count] = numpy_images[expo_args[1]]
                    image_tensor3[add_count] = numpy_images[expo_args[2]]
                        
                df_cod = df_cod.append({'age':age, 'path':file_paths[expo_args[0]], 'light': 1}, ignore_index=True)
                df_cod = df_cod.append({'age':age, 'path':file_paths[expo_args[1]], 'light': 2}, ignore_index=True)
                df_cod = df_cod.append({'age':age, 'path':file_paths[expo_args[2]], 'light': 3}, ignore_index=True)
                add_count += 1                
                
print("error_count:"+str(error_count))



[0.0125, 0.00625, 0.025] :['/gpfs/gpfs0/deep/data/Savannah_Professional_Practice2021_06_10_21/CodOtholiths-MachineLearning/Savannah_Professional_Practice/2013/70021/Nr09_age02/IMG_0157.JPG', '/gpfs/gpfs0/deep/data/Savannah_Professional_Practice2021_06_10_21/CodOtholiths-MachineLearning/Savannah_Professional_Practice/2013/70021/Nr09_age02/IMG_0158.JPG', '/gpfs/gpfs0/deep/data/Savannah_Professional_Practice2021_06_10_21/CodOtholiths-MachineLearning/Savannah_Professional_Practice/2013/70021/Nr09_age02/IMG_0159.JPG']
[1, 0, 2]
[0.0125, 0.00625, 0.025] :['/gpfs/gpfs0/deep/data/Savannah_Professional_Practice2021_06_10_21/CodOtholiths-MachineLearning/Savannah_Professional_Practice/2013/70021/Nr03_age07/IMG_0121.JPG', '/gpfs/gpfs0/deep/data/Savannah_Professional_Practice2021_06_10_21/CodOtholiths-MachineLearning/Savannah_Professional_Practice/2013/70021/Nr03_age07/IMG_0122.JPG', '/gpfs/gpfs0/deep/data/Savannah_Professional_Practice2021_06_10_21/CodOtholiths-MachineLearning/Savannah_Professiona

In [ ]:
# Error in folder:
# /scratch/disk2/Otoliths/codotoliths_erlend/CodOtholiths-MachineLearning/Savannah_Professional_Practice/2015/70117/nr 04 age_02/IMG_0020.JPG
#_0
#/scratch/disk2/Otoliths/codotoliths_erlend/CodOtholiths-MachineLearning/Savannah_Professional_Practice/2015/70117/Nr03age_02
#/scratch/disk2/Otoliths/codotoliths_erlend/CodOtholiths-MachineLearning/Savannah_Professional_Practice/2015/70331
def read_jpg_cods(B4_input_shape = (380, 380, 3), max_dataset_size = 5180, loadImgToMem=False):
    '''
    reads one .jpg file in each folder in structure of folders
    returns tensor with images, and 1-1 correspondence with age
    '''
    #base_dir = '/test123/Savannah_Professional_Practice' #project/cod-otoliths
    #base_dir = '/scratch/disk2/Otoliths/codotoliths_erlend/CodOtholiths-MachineLearning/Savannah_Professional_Practice' #project/cod-otoliths
    base_dir = '/gpfs/gpfs0/deep/data/Savannah_Professional_Practice2021/'
    dirs = set() # to get just 1 jpg file from each folder
    df_cod = pd.DataFrame(columns=['age', 'path', 'ExposureTime'])

    image_tensor = np.empty(shape=(max_dataset_size,)+B4_input_shape)

    add_count = 0
    base_dirs_posix = Path(base_dir)
    for some_year_dir in base_dirs_posix.iterdir():
        print(some_year_dir)
        if not(some_year_dir.is_dir()):
            break
        for filename in Path(some_year_dir).glob('**/*.JPG'):
            filepath =str(filename)
            dirname = os.path.dirname(filepath)
            if ( dirname not in dirs and os.path.basename(filepath)[0] != '.' ):
                dirs.add(dirname)
                begin_age = filepath.lower().find('age')
                print(filepath)
                age = filepath[begin_age+3:begin_age+5]
                age = int(age)

                if loadImgToMem==True:
                    pil_img = load_img(filepath, target_size=B4_input_shape, grayscale=False)
                    array_img = img_to_array(pil_img, data_format='channels_last')
                    image_tensor[add_count] = array_img
                    
                    

                df_cod = df_cod.append({'age':age, 'path':filepath, 'light'}, ignore_index=True)
                #df_cod = df_cod.append({'age':age, 'path':filepath+'2'}, ignore_index=True)
                add_count += 1
                #print(add_count)

    print(add_count)
    age = df_cod.age.values
    if loadImgToMem == True:
        return image_tensor, age
    else:
        return df_cod, age


In [ ]:
#adadf

In [4]:
loadImgToMem=True
os.environ["CUDA_VISIBLE_DEVICES"]="0"
tensorboard_path= './tensorboard_test2'
checkpoint_path = './checkpoints_test2/cod_oto_efficientnetBBB.{epoch:03d}-{val_loss:.2f}.hdf5'
a_batch_size = 16
B4_input_shape = (380, 380, 3)
B5_input_shape = (456, 456, 3)

new_shape = B4_input_shape

#new_shape = B5_input_shape
#B4_input_shape = B5_input_shape

image_tensor, age = read_jpg_cods(B4_input_shape, max_dataset_size = 5180, loadImgToMem=True)

/gpfs/gpfs0/deep/data/Savannah_Professional_Practice2021/2018
/gpfs/gpfs0/deep/data/Savannah_Professional_Practice2021/2018/70360/Nr06_age07/IMG_0033.JPG
/gpfs/gpfs0/deep/data/Savannah_Professional_Practice2021/2018/70360/Nr13_age09/IMG_0078.JPG
/gpfs/gpfs0/deep/data/Savannah_Professional_Practice2021/2018/70360/Nr10_age07/IMG_0059.JPG
/gpfs/gpfs0/deep/data/Savannah_Professional_Practice2021/2018/70360/Nr05_age03/IMG_0025.JPG
/gpfs/gpfs0/deep/data/Savannah_Professional_Practice2021/2018/70360/Nr07_age04/IMG_0039.JPG
/gpfs/gpfs0/deep/data/Savannah_Professional_Practice2021/2018/70360/Nr02_age05/IMG_0012.JPG
/gpfs/gpfs0/deep/data/Savannah_Professional_Practice2021/2018/70360/Nr11_age03/IMG_0065.JPG
/gpfs/gpfs0/deep/data/Savannah_Professional_Practice2021/2018/70360/Nr09_age08/IMG_0052.JPG
/gpfs/gpfs0/deep/data/Savannah_Professional_Practice2021/2018/70360/Nr03_age07/IMG_0015.JPG
/gpfs/gpfs0/deep/data/Savannah_Professional_Practice2021/2018/70360/Nr04_age02/IMG_0021.JPG
/gpfs/gpfs0/deep/d

In [ ]:
image_tensor[5178,:,:,0]

In [ ]:
image_tensor.shape

### Model config

In [7]:
train_datagen = ImageDataGenerator(
    zca_whitening=False,
    width_shift_range=0.2,
    height_shift_range=0.2, #20,
    #zoom_range=[0.5,1.0],
    rotation_range=360,
    horizontal_flip=False,
    vertical_flip=True,
    rescale=1./255)

rgb_efficientNetB4 = tf.keras.applications.EfficientNetB4(include_top=False, weights='imagenet', input_shape=B4_input_shape, classes=2)
z = dense1_linear_output(  rgb_efficientNetB4)
    
learning_rate=0.00005
learning_rate=0.006
adam = optimizers.Adam(learning_rate=learning_rate)

early_stopper = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=40, verbose=0,
                                      mode='min',restore_best_weights=True)

plateau = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=13, 
                                               verbose=0,mode='min')
        
def binary_accuracy_for_regression(y_true, y_pred):
    return K.mean(K.equal(y_true, K.round(y_pred)), axis=-1)

classWeight = None  

#Tensorflow 2.2 - wrap generator in tf.data.Dataset
def callGen():
    return train_datagen.flow(train_rb_imgs, train_age, batch_size=a_batch_size)

train_dataset = tf.data.Dataset.from_generator(callGen, 
                                   (tf.float32, tf.float32)).shuffle(
                                        128, reshuffle_each_iteration=True).repeat()

### KFold out-of-Fold split

In [8]:
train_idx, val_idx, test_idx = train_validate_test_split( range(0, len(image_tensor)) )
train_idx_oof = train_idx + val_idx
val_idx = None

train_rb_imgs = np.empty(shape=(len(train_idx),)+B4_input_shape)
train_age = []
for i in range(0, len(train_idx)):
    train_rb_imgs[i] = image_tensor[train_idx[i]]
    train_age.append(age[train_idx[i]])
    
train_rb_imgs2 = image_tensor[train_idx]
print(train_rb_imgs.shape)
print(train_rb_imgs2.shape)

(3626, 380, 380, 3)
(3626, 380, 380, 3)


In [ ]:
from sklearn.model_selection import KFold
# K-fold
a_seed=2021
kfold = KFold(n_splits = 5, random_state = a_seed, shuffle = True)
train_oof = image_tensor[train_idx_oof,:,:]
train_oof.shape
#train_idx_oof = tr
#train_img_tensor = 
#for fold, (trn_ind, val_ind) in enumerate(kfold.split(train)):

In [ ]:
print(age.shape)
age2 = np.vstack(age)
print(age2.shape)

In [ ]:
the_fold = 0
for fold, (trn_ind, val_ind) in enumerate(kfold.split(age)):
    X_train, X_val = image_tensor[trn_ind,:,:,:], image_tensor[val_ind,:,:,:]
    y_train, y_val = age[trn_ind], age[val_ind]
    print(X_train.shape)
    print(y_train.shape)
    val_rb_imgs = np.multiply(X_train, 1./255)
    test_rb_imgs = np.multiply(test_rb_imgs, 1./255)
    
    cod = Model(inputs=rgb_efficientNetB4.input, outputs=z)
    cod.compile(loss='mse', optimizer=adam, metrics=['mse', 'mape', binary_accuracy_for_regression] )
    tensorboard, checkpointer = get_checkpoint_tensorboard(tensorboard_path, checkpoint_path)
    
    for layer in cod.layers:
        layer.trainable = True
    
    print("training the fold:"+str(the_fold))
    the_fold += 1
    history_callback = cod.fit(train_dataset ,
        steps_per_epoch=1600,
        epochs=1,
        callbacks=[early_stopper, plateau, tensorboard, checkpointer],
        #validation_data = val_dataset,
        validation_data = (val_rb_imgs, val_age),
        class_weight=classWeight)

    #test_metrics = cod.evaluate_generator(test_dataset)
    test_metrics = cod.evaluate(x=test_rb_imgs, y=test_age)
    print("test metric:"+str(cod.metrics_names))
    print("test metrics:"+str(test_metrics))

    print("precision, recall, f1")
    y_pred_test = cod.predict(test_rb_imgs, verbose=1)
    y_pred_test_bool = np.argmax(y_pred_test, axis=1)
    y_true_bool = np.argmax(test_age, axis=1)
    #np.argmax inverse of to_categorical
    argmax_test = np.argmax(test_age, axis=1)
    unique, counts = np.unique(argmax_test, return_counts=True)
    print("test ocurrence of each class:"+str(dict(zip(unique, counts))))

    print("cslassification_report")
    print(classification_report(y_true_bool, y_pred_test_bool))
    print("confusion matrix")
    print(str(confusion_matrix(y_true_bool, y_pred_test_bool)))


(4144, 380, 380, 3)
(4144,)


In [ ]:
#loading img from memory

train_rb_imgs = np.empty(shape=(len(train_idx),)+B4_input_shape)
train_age = []
for i in range(0, len(train_idx)):
    train_rb_imgs[i] = image_tensor[train_idx[i]]
    train_age.append(age[train_idx[i]])

val_rb_imgs = np.empty(shape=(len(val_idx),)+B4_input_shape)
val_age = []
for i in range(0, len(val_idx)):
    val_rb_imgs[i] = image_tensor[val_idx[i]]
    val_age.append(age[val_idx[i]])

test_rb_imgs = np.empty(shape=(len(test_idx),)+B4_input_shape)
test_age = []
for i in range(0, len(test_idx)):
    test_rb_imgs[i] = image_tensor[test_idx[i]]
    test_age.append(age[test_idx[i]])

train_age = np.vstack(train_age)
val_age = np.vstack(val_age)
test_age = np.vstack(test_age)
age = np.vstack(age)

val_rb_imgs = np.multiply(val_rb_imgs, 1./255)
test_rb_imgs = np.multiply(test_rb_imgs, 1./255)

In [ ]:


#Tensorflow 2.2 - wrap generator in tf.data.Dataset
def callGen():
    return train_datagen.flow(train_rb_imgs, train_age, batch_size=a_batch_size)

def callGen_df():
    return train_datagen.flow_from_dataframe(dataframe=img_df.iloc[train_idx],
        directory=None, #absolute path 
        x_col="path",
        y_col="age",
        batch_size=32,
        seed=42,
        shuffle=False,
        class_mode="raw",  #int
        target_size=new_shape[0:2])

def callGen_df_val():
    return train_datagen.flow_from_dataframe(dataframe=img_df.iloc[val_idx],
        directory=None, 
        x_col="path",
        y_col="age",
        batch_size=32,
        seed=42,
        shuffle=False,
        class_mode="raw", 
        target_size=new_shape[0:2])

def callGen_df_test():
    return test_datagen.flow_from_dataframe(dataframe=img_df.iloc[test_idx],
        directory=None, 
        x_col="path",
        y_col="age",
        batch_size=32,
        seed=42,
        shuffle=False,
        class_mode="raw",  
        target_size=new_shape[0:2])



train_dataset=None
val_dataset=None
test_dataset=None
if loadImgToMem==True:
    train_dataset = tf.data.Dataset.from_generator(callGen, (tf.float32, tf.float32)).shuffle(128, reshuffle_each_iteration=True).repeat()
else: 
    train_dataset = tf.data.Dataset.from_generator(callGen_df, (tf.float32, tf.float32)).shuffle(128, reshuffle_each_iteration=False)
    val_dataset = tf.data.Dataset.from_generator(callGen_df_val, (tf.float32, tf.float32)).shuffle(128, reshuffle_each_iteration=False)
    test_dataset = tf.data.Dataset.from_generator(callGen_df_test, (tf.float32, tf.float32)).shuffle(128, reshuffle_each_iteration=False)

#history_callback = cod.fit(x=image_tensor,y=age, steps_per_epoch=1600, epochs=1)
#history_callback = cod.fit(callGen(), steps_per_epoch=1600, epochs=1)





In [ ]:
    ############## salmon_scales
    ''
    new_shape = (380, 380, 3)
    age_cod = age
    rb_imgs, all_sea_age, all_smolt_age, all_farmed_class, all_spawn_class, all_filenames = load_xy()
    uten_ukjent = len(all_sea_age) - all_sea_age.count(-1.0)
    rb_imgs2 = np.empty(shape=(uten_ukjent,)+new_shape)
    unique, counts = np.unique(all_sea_age, return_counts=True)
    print("age distrib:"+str( dict(zip(unique, counts)) ))

    all_sea_age2 = []
    found_count = 0
    all_filenames2 = []
    for i in range(0, len(all_sea_age)):
        if all_sea_age[i] > -1:
            rb_imgs2[found_count] = rb_imgs[i]
            all_sea_age2.append(all_sea_age[i])
            found_count += 1
            all_filenames2.append(all_filenames[i])

    assert found_count == uten_ukjent

    age_scales = all_sea_age2
    rb_imgs = rb_imgs2

    age_scales = np.vstack(age_scales)

    train_datagen_scales = ImageDataGenerator(
        zca_whitening=False,
        width_shift_range=0.2,
        height_shift_range=0.2, #20,
        #zoom_range=[0.5,1.0],
        rotation_range=360,
        horizontal_flip=False,
        vertical_flip=True,
        rescale=1./255)

    train_generator_scales = train_datagen_scales.flow(rb_imgs, age_scales, batch_size= a_batch_size)
    history_callback_scales = cod.fit(train_generator_scales,
        steps_per_epoch=1000,
        epochs=20,
        #callbacks=[early_stopper, tensorboard, checkpointer],
        #validation_data= (val_rb_imgs, val_age),
        class_weight=classWeight)
    ######################



In [ ]:

    K.set_value(cod.optimizer.learning_rate, 0.00001)
    print("Learning rate before second fit:", cod.optimizer.learning_rate.numpy())

    history_callback = cod.fit(train_dataset ,
        steps_per_epoch=1600,
        epochs=150,
        callbacks=[early_stopper, tensorboard, checkpointer],
        #validation_data = val_dataset,
        validation_data = (val_rb_imgs, val_age),
        class_weight=classWeight)

    #test_metrics = cod.evaluate_generator(test_dataset)
    test_metrics = cod.evaluate(x=test_rb_imgs, y=test_age)
    print("test metric:"+str(cod.metrics_names))
    print("test metrics:"+str(test_metrics))

    print("precision, recall, f1")
    y_pred_test = cod.predict(test_rb_imgs, verbose=1)
    y_pred_test_bool = np.argmax(y_pred_test, axis=1)
    y_true_bool = np.argmax(test_age, axis=1)
    #np.argmax inverse of to_categorical
    argmax_test = np.argmax(test_age, axis=1)
    unique, counts = np.unique(argmax_test, return_counts=True)
    print("test ocurrence of each class:"+str(dict(zip(unique, counts))))

    print("cslassification_report")
    print(classification_report(y_true_bool, y_pred_test_bool))
    print("confusion matrix")
    print(str(confusion_matrix(y_true_bool, y_pred_test_bool)))




